In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Modelos

In [6]:
!pip install langchain_openai langchain_core

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [4]:
response = llm.invoke("Olá")

print(response)

content='Olá! Como posso ajudá-lo hoje?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_48196bc67a', 'finish_reason': 'stop', 'logprobs': None} id='run-b1583581-c407-459f-be9b-109213b57deb-0' usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16}


In [5]:
response.content

'Olá! Como posso ajudá-lo hoje?'

## Prompts

### Templates Simples

In [9]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

#Runnable
prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: Artificial Intelligence is the future!')]


In [10]:
response = llm.invoke(prompt)

print(response.content)

Inteligência Artificial é o futuro!


### Templates de Mensagens

In [11]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

# messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

content='Olá, como você está?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 36, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'stop', 'logprobs': None} id='run-544e3b75-1a6b-41b6-a2ff-3d38d5a1cf83-0' usage_metadata={'input_tokens': 36, 'output_tokens': 6, 'total_tokens': 42}


In [12]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [13]:
prompt = prompt_template.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "português",
    "texto": "Hello, how are you?"
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas.'), HumanMessage(content='Hello, how are you?')]


In [14]:
response = llm.invoke(prompt)

print(response.content)

Olá, como você está?


### Parsers

In [15]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [16]:
output = parser.invoke(response)

print(output)

Olá, como você está?


## Encadeamento

In [17]:
chain = prompt_template | llm | parser

In [18]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [19]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [20]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

¡Las playas de Recife tienen tiburones!


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [21]:
prompt_template = ChatPromptTemplate.from_template("Faça uma piada sobre {topico}")

chain = prompt_template | llm | parser
chain.invoke({"topico": "futebol"})

'Por que o jogador de futebol levou um lápis para o jogo?\n\nPorque ele queria fazer um gol de letra! ⚽✏️'

### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [22]:
prompt_template = ChatPromptTemplate.from_template("Classifique o sentimento de um {texto} entre positivo, negativo ou neutro")

chain = prompt_template | llm | parser
chain.invoke({"texto": "Eu estou muito feliz. Hoje o dia vai ser maravilhoso"})

'A frase "Eu estou muito feliz. Hoje o dia vai ser maravilhoso" expressa um sentimento positivo.'

### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [24]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um programador que converte descrições de funções em códigos Python"),
        ("user", "{texto}")
    ]
)

chain = prompt_template | llm | parser
result = chain.invoke({"texto": "Uma função que calcule a multiplicação de dois numeros"})
print(result)

Aqui está uma função em Python que calcula a multiplicação de dois números:

```python
def multiplicar(num1, num2):
    return num1 * num2

# Exemplo de uso:
resultado = multiplicar(5, 3)
print(resultado)  # Saída: 15
```

Essa função `multiplicar` recebe dois argumentos, `num1` e `num2`, e retorna o produto deles. Você pode usar a função passando os números que deseja multiplicar.


### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [27]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Explique de forma simplificada o topico geral fornecido e em seguida traduza o topico geral para a língua Inglês"),
        ("user", "{texto}")
    ]
)

chain = prompt_template | llm | parser
result = chain.invoke({"texto": "Um jogador estará impedido quando estiver no campo de ataque e à frente do último adversário (menos o goleiro). Não há impedimento do jogador que estiver em seu campo de defesa ou quando há pelo menos dois adversário a sua frente."})
print(result)

O tópico geral fala sobre a regra de impedimento no futebol. Um jogador é considerado impedido se estiver no campo de ataque e à frente do último defensor, exceto o goleiro. No entanto, um jogador não está impedido se estiver em seu próprio campo de defesa ou se houver pelo menos dois defensores à sua frente.

Tradução para o inglês:
A player will be offside when they are in the attacking half and ahead of the last opponent (not including the goalkeeper). There is no offside for a player who is in their own defensive half or when there are at least two opponents in front of them.


### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [29]:
prompt_template = ChatPromptTemplate.from_template("Responda uma {pergunta} sobre o CESAR School")

chain = prompt_template | llm | parser
chain.invoke({"pergunta": "Quando foi fundado?"})

'O CESAR School foi fundado em 2015. Ele é uma instituição de ensino que faz parte do CESAR (Centro de Estudos e Sistemas Avançados do Recife), focando na formação de profissionais nas áreas de tecnologia e inovação. Se precisar de mais informações sobre o CESAR School ou suas atividades, fique à vontade para perguntar!'